In [1]:
import pandas as pd
import numpy as np

In [11]:
actuals_df = pd.read_csv("../../Data/Actuals.csv")
forecast_df = pd.read_csv("../../Data/Analyst Forecast.csv")

In [12]:
actuals_df.head(10)

,TICKER,OFTIC,CNAME,PENDS,MEASURE,PDICITY,ANNDATS,VALUE
0,003H,PAYC,PAYCOM SOFTWARE,2014-06-30,EPS,QTR,2014-08-05,0.04
1,003H,PAYC,PAYCOM SOFTWARE,2014-09-30,EPS,QTR,2014-11-04,0.05
2,003H,PAYC,PAYCOM SOFTWARE,2014-12-31,EPS,QTR,2015-02-10,0.06
3,003H,PAYC,PAYCOM SOFTWARE,2015-03-31,EPS,QTR,2015-05-06,0.12
4,003H,PAYC,PAYCOM SOFTWARE,2015-06-30,EPS,QTR,2015-08-04,0.10
5,003H,PAYC,PAYCOM SOFTWARE,2015-09-30,EPS,QTR,2015-11-03,0.08
6,003H,PAYC,PAYCOM SOFTWARE,2015-12-31,EPS,QTR,2016-02-09,0.10
7,003H,PAYC,PAYCOM SOFTWARE,2016-03-31,EPS,QTR,2016-05-03,0.33
8,003H,PAYC,PAYCOM SOFTWARE,2016-06-30,EPS,QTR,2016-08-02,0.21
9,003H,PAYC,PAYCOM SOFTWARE,2016-09-30,EPS,QTR,2016-11-01,0.15


In [13]:
forecast_df.head(10)

,TICKER,OFTIC,CNAME,ALYSNAM,VALUE,AMASKCD,ANNDATS
0,AT1,A,AGILENT TECH,PAI A,32.0,105573,2010-01-29
1,AT1,A,AGILENT TECH,STEIN W,35.0,117325,2010-02-08
2,AT1,A,AGILENT TECH,STEIN W,36.0,117325,2010-02-14
3,AT1,A,AGILENT TECH,GROBERG J,31.0,110441,2010-02-12
4,AT1,A,AGILENT TECH,PAI A,34.0,105573,2010-02-12
5,AT1,A,AGILENT TECH,BUTLER C,36.0,20011,2010-02-16
6,AT1,A,AGILENT TECH,DRAY D,34.0,56596,2010-02-16
7,AT1,A,AGILENT TECH,DOUGLASS M,37.0,127201,2010-02-16
8,AT1,A,AGILENT TECH,GROBERG J,38.0,110441,2010-03-05
9,AT1,A,AGILENT TECH,STEIN W,40.0,117325,2010-03-19


In [14]:
actuals_df.sort_values(by=['OFTIC', 'ANNDATS'], inplace=True)

In [15]:
first_announcement_dates = actuals_df.groupby(['OFTIC', 'MEASURE'])['PENDS'].first().reset_index()

In [16]:
print(first_announcement_dates)

      OFTIC MEASURE       PENDS
0         A     BPS  2010-01-31
1         A     CPS  2010-01-31
2         A     CPX  2010-01-31
3         A     CSH  2010-01-31
4         A     DPS  2010-01-31
...     ...     ...         ...
11020   ZTS     OPR  2013-03-31
11021   ZTS     PRE  2013-03-31
11022   ZTS     ROA  2013-03-31
11023   ZTS     ROE  2013-03-31
11024   ZTS     SAL  2013-03-31

[11025 rows x 3 columns]


In [17]:
#min period end date'i 2015 sonrası olan yani 2015'ten sonra başlayan
actuals_df['PENDS'] = pd.to_datetime(actuals_df['PENDS']) #.year kullanabilmek için datetime çevir

min_pend = actuals_df.groupby('OFTIC')['PENDS'].min()

# Filter companies with a minimum announce date after 2015
companies_to_keep = min_pend[min_pend.dt.year <= 2015].reset_index()['OFTIC']

# Create a new DataFrame with only the data for the selected companies
actuals_filtered = actuals_df[actuals_df['OFTIC'].isin(companies_to_keep)]

In [18]:
actuals_filtered['OFTIC'].unique

<bound method Series.unique of 2310        A
71193       A
71247       A
71301       A
71355       A
         ... 
518254    ZTS
518295    ZTS
518336    ZTS
518377    ZTS
518418    ZTS
Name: OFTIC, Length: 506917, dtype: object>

In [19]:
actuals_df['OFTIC'].unique

<bound method Series.unique of 2310        A
71193       A
71247       A
71301       A
71355       A
         ... 
518254    ZTS
518295    ZTS
518336    ZTS
518377    ZTS
518418    ZTS
Name: OFTIC, Length: 519504, dtype: object>

In [20]:
#2015'ten önceki data
actuals_filtered_ = actuals_filtered[actuals_filtered['PENDS'].dt.year >= 2015]

In [21]:
actuals_filtered_.shape

(320691, 8)

In [22]:
unique_day_month = actuals_filtered_['PENDS'].dt.strftime('%d-%m').unique()

# Print the unique day-month combinations
for date in unique_day_month:
    print(date)

31-01
30-04
31-07
31-10
31-03
30-06
30-09
31-12
28-02
31-05
31-08
30-11
29-02


In [23]:
def map_to_quarterly(date):
    if date.month >= 1 and date.month <= 3:
        return pd.Timestamp(date.year, 3, 31)
    elif date.month >= 4 and date.month <= 6:
        return pd.Timestamp(date.year, 6, 30)
    elif date.month >= 7 and date.month <= 9:
        return pd.Timestamp(date.year, 9, 30)
    else:
        return pd.Timestamp(date.year, 12, 31)

# Apply the mapping function to create the 'PENDS_Quarterly' column
actuals_filtered_['PENDS_Quarterly'] = actuals_filtered_['PENDS'].apply(map_to_quarterly)
print(actuals_filtered_)

       TICKER OFTIC    CNAME      PENDS MEASURE PDICITY     ANNDATS  \
2330      AT1     A  AGILENT 2015-01-31     EPS     QTR  2015-02-17   
71213     AT1     A  AGILENT 2015-01-31     BPS     QTR  2015-02-17   
71267     AT1     A  AGILENT 2015-01-31     CPS     QTR  2015-02-17   
71321     AT1     A  AGILENT 2015-01-31     CPX     QTR  2015-02-17   
71375     AT1     A  AGILENT 2015-01-31     CSH     QTR  2015-02-17   
...       ...   ...      ...        ...     ...     ...         ...   
518254   ZOTS   ZTS   ZOETIS 2023-03-31     OPR     QTR  2023-05-04   
518295   ZOTS   ZTS   ZOETIS 2023-03-31     PRE     QTR  2023-05-04   
518336   ZOTS   ZTS   ZOETIS 2023-03-31     ROA     QTR  2023-05-04   
518377   ZOTS   ZTS   ZOETIS 2023-03-31     ROE     QTR  2023-05-04   
518418   ZOTS   ZTS   ZOETIS 2023-03-31     SAL     QTR  2023-05-04   

            VALUE PENDS_Quarterly  
2330       0.4100      2015-03-31  
71213     12.6101      2015-03-31  
71267     -0.0600      2015-03-31  
713

/tmp/ipykernel_4645/1601742150.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actuals_filtered_['PENDS_Quarterly'] = actuals_filtered_['PENDS'].apply(map_to_quarterly)


In [24]:
unique_day_month = actuals_filtered_['PENDS_Quarterly'].dt.strftime('%d-%m').unique()

# Print the unique day-month combinations
for date in unique_day_month:
    print(date)

31-03
30-06
30-09
31-12


In [25]:
forecast_df['ANNDATS'] = pd.to_datetime(forecast_df['ANNDATS'])

# Define a function to calculate the PENDS_Quarterly
def calculate_pends_quarterly(anndat):
    if anndat.month >= 1 and anndat.month <= 3:
        return pd.Timestamp(anndat.year - 1, 12, 31)
    elif anndat.month >= 4 and anndat.month <= 6:
        return pd.Timestamp(anndat.year, 3, 31)
    elif anndat.month >= 7 and anndat.month <= 9:
        return pd.Timestamp(anndat.year, 6, 30)
    else:
        return pd.Timestamp(anndat.year, 9, 30)

# Apply the function to create the 'PENDS_Quarterly' column
forecast_df['PENDS_Quarterly'] = forecast_df['ANNDATS'].apply(calculate_pends_quarterly)

# Print the resulting DataFrame
print(forecast_df)

       TICKER OFTIC         CNAME               ALYSNAM  VALUE  AMASKCD  \
0         AT1     A  AGILENT TECH  PAI                A   32.0   105573   
1         AT1     A  AGILENT TECH  STEIN              W   35.0   117325   
2         AT1     A  AGILENT TECH  STEIN              W   36.0   117325   
3         AT1     A  AGILENT TECH  GROBERG            J   31.0   110441   
4         AT1     A  AGILENT TECH  PAI                A   34.0   105573   
...       ...   ...           ...                   ...    ...      ...   
404815   ZOTS   ZTS        ZOETIS  CHESNEY            S  225.0   130921   
404816   ZOTS   ZTS        ZOETIS  RICH               N  204.0   157521   
404817   ZOTS   ZTS        ZOETIS  WESTENBERG         D  210.0   185530   
404818   ZOTS   ZTS        ZOETIS  MAJUMDAR           I  190.0   194536   
404819   ZOTS   ZTS        ZOETIS  HARDY              S  174.0   196139   

          ANNDATS PENDS_Quarterly  
0      2010-01-29      2009-12-31  
1      2010-02-08      2009

In [26]:
#min period end date'i 2015 sonrası olan yani 2015'ten sonra başlayan
forecast_df['PENDS_Quarterly'] = pd.to_datetime(forecast_df['PENDS_Quarterly']) #.year kullanabilmek için datetime çevir

min_pend = forecast_df.groupby('OFTIC')['PENDS_Quarterly'].min()

# Filter companies with a minimum announce date after 2015
companies_to_keep = min_pend[min_pend.dt.year <= 2015].reset_index()['OFTIC']

# Create a new DataFrame with only the data for the selected companies
forecast_filtered = forecast_df[actuals_df['OFTIC'].isin(companies_to_keep)]

/tmp/ipykernel_4645/968291195.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  forecast_filtered = forecast_df[actuals_df['OFTIC'].isin(companies_to_keep)]


In [27]:
#2015'ten önceki data
forecast_filtered_ = forecast_filtered[forecast_filtered['PENDS_Quarterly'].dt.year >= 2015].reset_index()

In [28]:
forecast_filtered_.head(15)

,index,TICKER,OFTIC,CNAME,ALYSNAM,VALUE,AMASKCD,ANNDATS,PENDS_Quarterly
0,231,AT1,A,AGILENT,BEUCHAW S,47.0,147830,2015-04-10,2015-03-31
1,232,AT1,A,AGILENT,MUKEN R,46.0,112085,2015-04-13,2015-03-31
2,233,AT1,A,AGILENT,PETERSON T,48.0,107607,2015-04-21,2015-03-31
3,234,AT1,A,AGILENT,MUKEN R,45.0,112085,2015-05-18,2015-03-31
4,235,AT1,A,AGILENT,DE BRUIN D,46.0,86687,2015-05-28,2015-03-31
5,236,AT1,A,AGILENT,COUILLARD B,48.0,138700,2015-05-29,2015-03-31
6,237,AT1,A,AGILENT,MUKEN R,44.0,112085,2015-08-17,2015-06-30
7,238,AT1,A,AGILENT,RO I,41.0,99673,2015-08-17,2015-06-30
8,239,AT1,A,AGILENT,RO I,40.0,99673,2015-08-27,2015-06-30
9,240,AT1,A,AGILENT,MUKEN R,42.5,112085,2015-09-09,2015-06-30


In [29]:
grouped = forecast_filtered_.groupby(['OFTIC', 'PENDS_Quarterly'])

mean_values = grouped['VALUE'].mean()
std_deviation = grouped['VALUE'].std()

mean_values = mean_values.reset_index()
std_deviation = std_deviation.reset_index()

# Rename the columns for merging
mean_values = mean_values.rename(columns={'VALUE': 'MEAN'})
std_deviation = std_deviation.rename(columns={'VALUE': 'STDEV'})

# Merge the mean and standard deviation DataFrames with the original DataFrame
forecast_filtered_ = pd.merge(forecast_filtered_, mean_values, on=['OFTIC', 'PENDS_Quarterly'], how='left')
forecast_filtered_ = pd.merge(forecast_filtered_, std_deviation, on=['OFTIC', 'PENDS_Quarterly'], how='left')


In [41]:
forecast_filtered_[forecast_filtered_["OFTIC"] == "A"]

,index,TICKER,OFTIC,CNAME,ALYSNAM,VALUE,AMASKCD,ANNDATS,PENDS_Quarterly,MEAN,STDEV
0,231,AT1,A,AGILENT,BEUCHAW S,47.0,147830,2015-04-10,2015-03-31,46.666667,1.211060
1,232,AT1,A,AGILENT,MUKEN R,46.0,112085,2015-04-13,2015-03-31,46.666667,1.211060
2,233,AT1,A,AGILENT,PETERSON T,48.0,107607,2015-04-21,2015-03-31,46.666667,1.211060
3,234,AT1,A,AGILENT,MUKEN R,45.0,112085,2015-05-18,2015-03-31,46.666667,1.211060
4,235,AT1,A,AGILENT,DE BRUIN D,46.0,86687,2015-05-28,2015-03-31,46.666667,1.211060
...,...,...,...,...,...,...,...,...,...,...,...
394,625,AT1,A,AGILENT,SERGOTT L,135.0,190764,2023-05-24,2023-03-31,144.331250,10.416443
395,626,AT1,A,AGILENT,COUILLARD B,150.0,138700,2023-05-24,2023-03-31,144.331250,10.416443
396,627,AT1,A,AGILENT,DE BRUIN D,144.0,86687,2023-05-24,2023-03-31,144.331250,10.416443
397,628,AT1,A,AGILENT,DALEY T,160.0,188760,2023-05-24,2023-03-31,144.331250,10.416443


In [31]:
print(forecast_filtered_.groupby(['OFTIC','PENDS_Quarterly'])['MEAN'].min())

OFTIC  PENDS_Quarterly
A      2015-03-31          46.666667
       2015-06-30          41.875000
       2015-09-30          43.590909
       2015-12-31          43.961538
       2016-03-31          49.000000
                             ...    
ZTS    2022-03-31         218.500000
       2022-06-30         207.000000
       2022-09-30         195.375000
       2022-12-31         203.581818
       2023-03-31         200.000000
Name: MEAN, Length: 15375, dtype: float64


merge yanlış row sayısı olması gerekenden çok fazla çıkıyor

In [32]:
merged_df = pd.merge(actuals_filtered_, forecast_filtered_, on=["PENDS_Quarterly", "OFTIC"], suffixes=("_actual", "_forecast"))

In [33]:
merged_df.head(10)

,TICKER_actual,OFTIC,CNAME_actual,PENDS,MEASURE,PDICITY,ANNDATS_actual,VALUE_actual,PENDS_Quarterly,index,TICKER_forecast,CNAME_forecast,ALYSNAM,VALUE_forecast,AMASKCD,ANNDATS_forecast,MEAN,STDEV
0,AT1,A,AGILENT,2015-01-31,EPS,QTR,2015-02-17,0.4100,2015-03-31,231,AT1,AGILENT,BEUCHAW S,47.0,147830,2015-04-10,46.666667,1.21106
1,AT1,A,AGILENT,2015-01-31,EPS,QTR,2015-02-17,0.4100,2015-03-31,232,AT1,AGILENT,MUKEN R,46.0,112085,2015-04-13,46.666667,1.21106
2,AT1,A,AGILENT,2015-01-31,EPS,QTR,2015-02-17,0.4100,2015-03-31,233,AT1,AGILENT,PETERSON T,48.0,107607,2015-04-21,46.666667,1.21106
3,AT1,A,AGILENT,2015-01-31,EPS,QTR,2015-02-17,0.4100,2015-03-31,234,AT1,AGILENT,MUKEN R,45.0,112085,2015-05-18,46.666667,1.21106
4,AT1,A,AGILENT,2015-01-31,EPS,QTR,2015-02-17,0.4100,2015-03-31,235,AT1,AGILENT,DE BRUIN D,46.0,86687,2015-05-28,46.666667,1.21106
5,AT1,A,AGILENT,2015-01-31,EPS,QTR,2015-02-17,0.4100,2015-03-31,236,AT1,AGILENT,COUILLARD B,48.0,138700,2015-05-29,46.666667,1.21106
6,AT1,A,AGILENT,2015-01-31,BPS,QTR,2015-02-17,12.6101,2015-03-31,231,AT1,AGILENT,BEUCHAW S,47.0,147830,2015-04-10,46.666667,1.21106
7,AT1,A,AGILENT,2015-01-31,BPS,QTR,2015-02-17,12.6101,2015-03-31,232,AT1,AGILENT,MUKEN R,46.0,112085,2015-04-13,46.666667,1.21106
8,AT1,A,AGILENT,2015-01-31,BPS,QTR,2015-02-17,12.6101,2015-03-31,233,AT1,AGILENT,PETERSON T,48.0,107607,2015-04-21,46.666667,1.21106
9,AT1,A,AGILENT,2015-01-31,BPS,QTR,2015-02-17,12.6101,2015-03-31,234,AT1,AGILENT,MUKEN R,45.0,112085,2015-05-18,46.666667,1.21106


In [25]:
merged_df.shape

(5546381, 18)